In [1]:
def zero_matrix(nrow, ncol):
    M = []
    while len(M) < nrow:
        M.append([]) # create a new row
        while len(M[-1]) < ncol:
            M[-1].append(0.0)
    return M

In [2]:
def zero_array_3d(time, row, col):
    M = zero_matrix(row, col)
    M_3d = [M]
    while len(M_3d) < time:
        M_3d.append(M)
    return M_3d

In [3]:
def repr_mat(A):
    return '{} {} {}'.format(len(A), len(A[0]), ' '.join(' '.join(str(v) for v in l) for l in A))

In [4]:
# HMM2 - Decoding
from math import log, inf

In [5]:
with open('hmm4_01.in') as inp:
    A_list = list(map(float, inp.readline().split()))
    B_list = list(map(float, inp.readline().split()))
    pi_list = list(map(float, inp.readline().split()))
    obs_list = list(map(int, inp.readline().split()))

In [6]:
N = int(A_list[0])
K = int(B_list[1])
T = obs_list[0]

In [7]:
# initial lambda

A = zero_matrix(N, N)
B = zero_matrix(N, K)
pi = pi_list[2:]
# pi = [pi_list[2: ]] # matrix type

obs = obs_list[1:]

In [8]:
for i in range(N):
    for j in range(N):
        A[i][j] = A_list[2 + N*i + j]
    for j in range(K):
        B[i][j] = B_list[2 + K*i + j]

In [9]:
# alpha-pass: ouput alphas as a T * N matrix, and coefficients c_t
def alpha_pass_scaled(A, B, pi, obs):
    # by log
    T = len(obs)
    N = len(A)
    alphas = zero_matrix(T, N)
    c = [0.0] * T
    alpha_tilde = [pi[i] * B[i][obs[0]] for i in range(N)]
    c[0] = 1.0 / sum(alpha_tilde)
    alphas[0] = [c[0] * alpha for alpha in alpha_tilde]
    for t in range(1, T):
        alpha_tilde = [sum(alphas[t-1][j] * A[j][i] * B[i][obs[t]] for j in range(N)) for i in range(N)]
        c[t] = 1.0 / sum(alpha_tilde)
        alphas[t] = [c[t] * alpha for alpha in alpha_tilde]
    return alphas, c

In [10]:
alphas, c = alpha_pass_scaled(A, B, pi, obs)

In [11]:
# beta-pass
def beta_pass_scaled(A, B, obs, c):
    T = len(obs)
    N = len(A)
    betas = zero_matrix(T, N)
    betas[T-1] = [c[T-1]] * N
    for t in range(T-2, -1, -1):
        betas[t] = [sum(betas[t+1][j] * B[j][obs[t+1]] * A[i][j] for j in range(N)) * c[t] for i in range(N) ]
    return betas

In [12]:
betas = beta_pass_scaled(A, B, obs, c)
print(betas[T-1])

[3.916627012142473, 3.916627012142473, 3.916627012142473, 3.916627012142473]


In [13]:
def learning_iter(A, B, pi, obs):
    N = len(A)
    K = len(B[0])
    T = len(obs)
    # calculate alpha, beta, di_gamma, gamma
    alphas, c = alpha_pass_scaled(A, B, pi, obs)
    betas = beta_pass_scaled(A, B, obs, c)
    # di_gamma and gamma are computed at each time
    # sum of di_gamma and gamma over time 0:T-1
    sum_t_di_gamma = zero_matrix(N, N)
    sum_t_gamma = [0.0] * N
    sum_O_k_count_times_gamma = zero_matrix(N, K) # numerator of b_j(k)
    new_pi = None
    for t in range(T-1):
        di_gamma_t = zero_matrix(N, N)
        gamma_t = [0.0] * N
        for i in range(N):
            for j in range(N):
                di_gamma_t[i][j] = betas[t+1][j] * A[i][j] * B[j][obs[t+1]] * alphas[t][i]
                gamma_t[i] += di_gamma_t[i][j]
                sum_t_di_gamma[i][j] += di_gamma_t[i][j]
            sum_t_gamma[i] += gamma_t[i]
            sum_O_k_count_times_gamma[i][obs[t]] += gamma_t[i]
        if t == 0:
            new_pi = gamma_t

    new_A = zero_matrix(N, N)
    new_B = zero_matrix(N, K)
    for i in range(N):
        for j in range(N):
            new_A[i][j] = sum_t_di_gamma[i][j] / sum_t_gamma[i]
        for k in range(K):
            new_B[i][k] = sum_O_k_count_times_gamma[i][k] / sum_t_gamma[i]
    return new_A, new_B, new_pi, c

In [17]:
def learning(A, B, pi, obs, MAX_ITER, EPS):
        old_log_prob = -inf
        for iteration in range(MAX_ITER):
            A, B, pi, c = learning_iter(A, B, pi, obs)
            log_prob = -sum(log(ci) for ci in c)
            if log_prob <= old_log_prob or abs(log_prob - old_log_prob) <= EPS:
                break
            old_log_prob = log_prob
        print("final log-prob:", log_prob)
        return A, B, pi

In [18]:
def main(A, B, pi, obs):
    MAX_ITER = 10
    EPS = 1e-4
    A, B, pi = learning(A, B, pi, obs, MAX_ITER, EPS)
    print(repr_mat(A))
    print(repr_mat(B))
    return

In [19]:
main(A, B, pi, obs)

final log-prob: -692.1630999374814
4 4 0.5437340476678884 0.4562659523242489 7.040043914768708e-12 8.229468725975176e-13 1.1366543995256129e-12 0.5061278115235298 0.49387218846893177 6.401129681917852e-12 5.984315504249304e-12 2.452837372382555e-12 0.5042030347959278 0.4957969651956348 0.4781221318399029 4.322098096648015e-12 1.7332349423097942e-12 0.5218778681540418
4 4 0.9996635475968267 0.00017198676093968984 9.759384357438374e-25 0.00016446564223387267 0.0001670812261743474 0.9996908215714922 0.00014209720233299196 1.910543723397437e-24 1.7732805042799e-24 0.0002150318388916758 0.9995773314548447 0.00020763670626387456 0.0001681113481092875 1.1848196397612162e-24 0.00013233852739057595 0.9996995501245005
